<a href="https://colab.research.google.com/github/arunm917/Climate-Action-Tool/blob/main/equation_solver_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading packages and files

In [ ]:
import numpy as np
import pandas as pd
import gdown
import regex as re
from scipy.optimize import minimize

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# downloading file from gdrive
output = 'equations'
file_id = '1qEMQT4lnXe0X2WgNZ2QHciH1lw_rDsgp' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qEMQT4lnXe0X2WgNZ2QHciH1lw_rDsgp
To: /content/equations
100%|██████████| 89.0/89.0 [00:00<00:00, 260kB/s]


DONE.


In [ ]:
# downloading file from gdrive
output = 'parameters'
file_id = '1qEVz2N1atZjRtPK4kGYbZp-bOIrctm2V' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qEVz2N1atZjRtPK4kGYbZp-bOIrctm2V
To: /content/parameters
100%|██████████| 212/212 [00:00<00:00, 613kB/s]


DONE.


In [ ]:
# downloading file from gdrive
output = 'variables'
file_id = '1qFxa_f1ApVT0BBi_ed826kZ9ombKqap5' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qFxa_f1ApVT0BBi_ed826kZ9ombKqap5
To: /content/variables
100%|██████████| 38.0/38.0 [00:00<00:00, 75.9kB/s]


DONE.


# Processing files

In [ ]:
parameters = pd.read_csv('parameters')

In [ ]:
parameters

,parameters,values,Status,Remark
0,phi1,3,Float,Parameter is part of the decision variable
1,phi2,2,fixed,Value is fixed
2,phi3,2,Correct,Paramete is changed by a small margin by min. ...


In [ ]:
variables_df = pd.read_csv('variables')
variables_df

,variables,values
0,x,0
1,y,0
2,z,0
3,w,0


In [ ]:
variables_list = list(variables_df['variables'])

In [ ]:
print(variables_list)

['x', 'y', 'z', 'w']


In [ ]:
with open('equations', 'r') as f:
    # Read the equations line by line
    eq_lines = f.readlines()

In [ ]:
print(eq_lines)

['eq1: x + y - (phi1 / phi2) * w - 10\n', 'eq2: x - y + phi2 * z - 3\n', 'eq3: phi3 * x + y - z + 6']


In [ ]:
# Create a list to store the equations
eq_list = []

# Loop through the equation lines
for eq_line in eq_lines:
    # Split the line into the equation name and the equation expression
    eq_name, eq_expr = eq_line.strip().split(':')
    # Convert the tuple of symbols to a single expression
    eq_list.append(eq_expr) 
    # Add the equation to the dictionary
print(eq_list)

[' x + y - (phi1 / phi2) * w - 10', ' x - y + phi2 * z - 3', ' phi3 * x + y - z + 6']


In [ ]:
# Creating dictionary for parameters and values
param_dict = dict(zip(parameters['parameters'],parameters['values']))
print(param_dict)

{'phi1': 3, 'phi2': 2, 'phi3': 2}


In [ ]:
# Substituting parameters in equation with their values
modified_list = []
for eq in eq_list:
  for key in param_dict:
    if key in eq:
        value = param_dict.get(key)
        eq = eq.replace(key, str(param_dict.get(key)))
  modified_list.append(eq)
print(modified_list)

[' x + y - (3 / 2) * w - 10', ' x - y + 2 * z - 3', ' 2 * x + y - z + 6']


In [ ]:
for item in modified_list:
  print(item)

 x + y - (3 / 2) * w - 10
 x - y + 2 * z - 3
 2 * x + y - z + 6


# Evaluating coefficients

In [ ]:
corrected_equations = []
expressions_inside_parentheses = []
for equation in modified_list:
  matches = re.findall(r'(\(.*?\))', equation)
  # print(matches)
  for expression in matches:
    # print(expression)
    eval_coeff = eval(expression)
    # print(eval_coeff)
    equation = equation.replace(expression, str(eval_coeff))
  # expressions_inside_parentheses.extend(matches)
  corrected_equations.append(equation)

print(corrected_equations)
# print(expressions_inside_parentheses)

[' x + y - 1.5 * w - 10', ' x - y + 2 * z - 3', ' 2 * x + y - z + 6']


In [ ]:
for item in corrected_equations:
  print(item)

 x + y - 1.5 * w - 10
 x - y + 2 * z - 3
 2 * x + y - z + 6


#Extracting coefficients to setup optimization problem

In [ ]:
equations_list = corrected_equations
# variables_list
coefficients_nested_list = []
constants_list = []

for equation in equations_list:
  coefficients_list = []
  # print('Equation:', equation)
  equation_split = equation.split()
  # print('Equation_split:', equation_split)
  i = 1
  for variable in variables_list:
    coeff = 0
    # print('i:',i)
    # print('variable:', variable)
    if variable in equation_split:
      variable_index = equation_split.index(variable)
      # print('variable_index:', variable_index)
      if i == 1:
        if variable_index == 0:
          coeff = 1
          coefficients_list.append(float(coeff))
        
        if variable_index == 1:
          coeff = -1
          coefficients_list.append(float(coeff))
        
        if variable_index == 2:
          coeff = equation_split[variable_index - 2]
          coefficients_list.append(float(coeff))

      else:
        if equation_split[variable_index - 1] == '-' or '+':
          # print(equation_split[variable_index - 1])
          sign = equation_split[variable_index - 1]
          if sign == '+':
            coeff = +1
            coefficients_list.append(float(coeff))
          if sign == '-':
            coeff = -1
            coefficients_list.append(float(coeff))

        if equation_split[variable_index - 1] == '*':
          # print(equation_split[variable_index - 1])
          sign = equation_split[variable_index - 3]
          number = equation_split[variable_index - 2]
          coeff_list = [sign, number]
          # print(coeff_list)
          coeff = float(''.join(coeff_list))
          coefficients_list.append(coeff)
          
      i += 1
    else:
      coeff = 0
      coefficients_list.append(float(coeff))

    
  coefficients_nested_list.append(coefficients_list)
  constant = [equation_split[-2], equation_split[-1]]
  constant = float(''.join(constant))
  constants_list.append(-1*constant)
  # print(constants_list)
  # print(coefficients_list)

print('A:', coefficients_nested_list)
print('b:', constants_list)

A: [[1.0, 1.0, 0.0, -1.5], [1.0, -1.0, 2.0, 0.0], [2.0, 1.0, -1.0, 0.0]]
b: [10.0, 3.0, -6.0]


# Solving the optimization problem

In [ ]:
# Define the matrix A and the vector b
# A = np.array([[1, 1, 0, -3], [1, -1, 2, 0], [2, 1, -1, 0]])
# b = np.array([10, 3, -6])
A = np.array(coefficients_nested_list)
b = np.array(constants_list)

In [ ]:
# Define the objective function
def objective_function(x):
    # print(np.linalg.norm(A.dot(x) - b, 2))
    return np.linalg.norm(A.dot(x) - b, 2)

In [ ]:
# Define the constraint
def constraint(x):
    return A.dot(x) - b

In [ ]:
# Define the initial guess for x
x0 = np.array([1, 1, 1, 1])


In [ ]:
# Define the optimization problem
problem = {'type': 'eq', 'fun': constraint}


In [ ]:
# Solve the optimization problem
solution = minimize(objective_function, x0, constraints=problem)

# Print the solution
print("Solution: ", solution.x)

Solution:  [-2.31368133  2.56840665  3.94104399 -6.49684978]


Non-linear function

In [ ]:
# Define the objective function
def objective_function(initial_guess):
    x, y, phi1, z = initial_guess
    eq1 = x + y - phi1 * z - 10
    eq2 = x - y + 2 * z - 3
    eq3 = 2 * x + y - z + 6
    return eq1**2 + eq2**2 + eq3**2

In [ ]:
# Initial guess for the decision variables
initial_guess = [0, 0, 0, 0]

# Bounds for the decision variables
bounds = [(-10, 10), (-10, 10), (-10, 10), (-10, 10)]

result = minimize(objective_function, initial_guess, bounds=bounds)

In [ ]:
# Extract the optimal solution
optimal_solution = result.x
print("Optimal solution:")
print("x =", optimal_solution[0])
print("y =", optimal_solution[1])
print("phi1 =", optimal_solution[2])
print("z =", optimal_solution[3])

Optimal solution:
x = -2.351620300290772
y = 2.7581015833364155
phi1 = -2.3659303995706917
z = 4.054860930459148
